# 0.1. imports

In [1]:
# i/o
import sys
import os
from pathlib import Path
import gzip
import pickle
# configuration
import yaml
# lca
import ecospold2matrix as e2m
import pymrio
#import brightway2 as bw
# type hints
from ecospold2matrix import ecospold2matrix
from pymrio import IOSystem
# data science
import pandas as pd
# deep copy
import copy

# 0.2. config

In [2]:
with open('../config.yaml', 'r') as filestream:
    config = yaml.load(filestream, Loader = yaml.FullLoader)

# 0.3. file paths

## 0.3.1. directories

In [3]:
%%capture
print(path_dir_data := os.path.join(Path.home(), config['path_dir_data']))
print(path_dir_data_raw := os.path.join(path_dir_data, config['path_dir_data_raw']))
# exiobase
print(path_dir_exiobase := os.path.join(path_dir_data_raw, 'IOT_2011_pxp'))

## 0.3.2. files

In [17]:
%%capture
print(path_file_impacts_S := os.path.join(path_dir_exiobase, 'impacts', 'S.txt'))
print(path_file_impacts_S_Y := os.path.join(path_dir_exiobase, 'impacts', 'S_Y.txt'))
print(path_file_impacts_M := os.path.join(path_dir_exiobase, 'impacts', 'M.txt'))
print(path_file_impacts_F := os.path.join(path_dir_exiobase, 'impacts', 'F.txt'))
print(path_file_impacts_F_Y := os.path.join(path_dir_exiobase, 'impacts', 'F_Y.txt'))

In [19]:
%%capture
print(path_file_satellite_S := os.path.join(path_dir_exiobase, 'satellite', 'S.txt'))
print(path_file_satellite_S_Y := os.path.join(path_dir_exiobase, 'satellite', 'S_Y.txt'))
print(path_file_satellite_M := os.path.join(path_dir_exiobase, 'satellite', 'M.txt'))
print(path_file_satellite_F := os.path.join(path_dir_exiobase, 'satellite', 'F.txt'))
print(path_file_satellite_F_Y := os.path.join(path_dir_exiobase, 'satellite', 'F_Y.txt'))

In [5]:
%%capture
print(path_file_C_IWP := os.path.join(path_dir_data_raw, 'exiobase_characterization', 'C_exio_IW_1.28_1.46.csv'))

# 1. load files

In [6]:
def read_csv(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = ',',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [7]:
def read_ecoinvent_txt(filepath: Path) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer = filepath,
        sep = '\t',
        engine = 'python',
        encoding = 'utf-8',
        index_col = None
    )
    return df

In [20]:
impacts_S = read_ecoinvent_txt(path_file_impacts_S)
impacts_S_Y = read_ecoinvent_txt(path_file_impacts_S_Y)
impacts_M = read_ecoinvent_txt(path_file_impacts_M)
impacts_F = read_ecoinvent_txt(path_file_impacts_F)
impacts_F_Y = read_ecoinvent_txt(path_file_impacts_F_Y)

In [21]:
satellite_S = read_ecoinvent_txt(path_file_satellite_S)
satellite_S_Y = read_ecoinvent_txt(path_file_satellite_S_Y)
satellite_M = read_ecoinvent_txt(path_file_satellite_M)
satellite_F = read_ecoinvent_txt(path_file_satellite_F)
satellite_F_Y = read_ecoinvent_txt(path_file_satellite_F_Y)

In [9]:
C_IWP = read_csv(path_file_C_IWP)

# 2. investigation
#### 2.1. `C_IWP`...up-to-date characterization factors for Exiobase (https://zenodo.org/record/4739143)

In [10]:
C_IWP

,Unnamed: 0,Taxes less subsidies on products purchased: Total,Other net taxes on production,"Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled","Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled","Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",Operating surplus: Consumption of fixed capital,Operating surplus: Rents on land,Operating surplus: Royalties on resources,Operating surplus: Remaining net operating surplus,...,Water Withdrawal Blue - Domestic - domestic Water Withdrawal Blue,Energy Carrier Net Total,Energy Carrier Net NENE,Energy Carrier Net NTRA,Energy Carrier Net TAVI,Energy Carrier Net TMAR,Energy Carrier Net TOTH,Energy Carrier Net TRAI,Energy Carrier Net TROA,Energy Carrier Net LOSS
0,"Climate change, ecosystem quality, long term (...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Climate change, ecosystem quality, short term ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Climate change, human health, long term (DALY)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Climate change, human health, short term (DALY)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Climate change, long term (kg CO2 eq (long))",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Climate change, short term (kg CO2 eq (short))",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Freshwater acidification (PDF.m2.yr),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Freshwater acidification (kg SO2 eq),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Freshwater ecotoxicity (CTUe),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Freshwater ecotoxicity, long term (PDF.m2.yr)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
len(C_IWP.columns.tolist()[1:])

1113

In [11]:
C_IWP.columns.tolist()[1:20]

['Taxes less subsidies on products purchased: Total',
 'Other net taxes on production',
 "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",
 'Operating surplus: Consumption of fixed capital',
 'Operating surplus: Rents on land',
 'Operating surplus: Royalties on resources',
 'Operating surplus: Remaining net operating surplus',
 'Employment: Low-skilled male',
 'Employment: Low-skilled female',
 'Employment: Medium-skilled male',
 'Employment: Medium-skilled female',
 'Employment: High-skilled male',
 'Employment: High-skilled female',
 'Employment hours: Low-skilled male',
 'Employment hours: Low-skilled female',
 'Employment hours: Medium-skilled male',
 'Employment hours: Medium-skilled female']

In [13]:
C_IWP['Unnamed: 0'].tolist()[-0:-10]

['Climate change, ecosystem quality, long term (PDF.m2.yr)',
 'Climate change, ecosystem quality, short term (PDF.m2.yr)',
 'Climate change, human health, long term (DALY)',
 'Climate change, human health, short term (DALY)',
 'Climate change, long term (kg CO2 eq (long))',
 'Climate change, short term (kg CO2 eq (short))',
 'Freshwater acidification (PDF.m2.yr)',
 'Freshwater acidification (kg SO2 eq)',
 'Freshwater ecotoxicity (CTUe)',
 'Freshwater ecotoxicity, long term (PDF.m2.yr)',
 'Freshwater ecotoxicity, short term (PDF.m2.yr)',
 'Freshwater eutrophication (PDF.m2.yr)',
 'Freshwater eutrophication (kg PO4 P-lim eq)',
 'Human toxicity cancer (CTUh)',
 'Human toxicity cancer, long term (DALY)',
 'Human toxicity cancer, short term (DALY)',
 'Human toxicity non cancer (CTUh)',
 'Human toxicity non-cancer, long term (DALY)',
 'Human toxicity non-cancer, short term (DALY)',
 'Marine acidification, long term (PDF.m2.yr)',
 'Marine acidification, short term (PDF.m2.yr)',
 'Marine eutro

#### 2.2. `F`...factors of production, stressors, impacts (https://zenodo.org/record/5589597)

In [22]:
impacts_F.iloc[10:20]

,region,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,AT.8,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
10,Respiratory effects on humans caused by organi...,0,0.531831905426,1.28692926827,0.427476795439,0.141828891826,0.090486571233,6.37015075904e-06,0.142779730382,0.448295013177,...,0.182593685632,0.155264359942,0.17347542763,0.0844667293525,0.100895994989,0.535105749915,1.20974417724,3.93514274329,0.0350348252911,0
11,Respiratory effects on humans caused by inorga...,0,993060.301927,1253894.97996,536717.08762,153378.6378,70939.240033,34.4850392872,109015.708649,888579.346808,...,38317965.5794,25076199.888,58099330.0262,18451629.2558,20639350.4651,80754222.7628,94466505.8864,156566741.684,562868.406186,0
12,Damages to human health caused by climate chan...,9.51138942895e-12,103.527648623,149.015585124,47.8492422359,34.889471455,17.0689359166,0.104446275838,12.8301240137,400.32111911,...,5646.52433295,247.063105301,69.444047218,531.867900129,165.604520594,100.179340652,654.461251351,1216.14860719,13.2559070245,0
13,Damage to Ecosystem Quality caused by ecotoxic...,0,992940.198051,1253692.62925,536648.458073,153354.230623,70930.6810247,34.4830629582,109000.296075,888457.397712,...,38318247.0791,25076317.0935,58099920.6916,18451746.5014,20639473.1395,80755013.2526,94467140.1656,156567959.616,562870.129002,0
14,Damage to Ecosystem Quality caused by the comb...,0,85996055.436,143029302.393,37390185.0982,37856563.0934,18065484.989,125757.320374,11221580.8745,419783256.764,...,17907226.8527,17153659.7587,16624653.7317,10587270.2926,11531498.2988,15778542.2103,254347064.406,93422435.6844,878234.811476,0
15,Carcinogenic effects on humans (E.E) | ECOINDI...,0,6.9314373997,14.3046937388,4.20033136288,1.72695038614,0.510191775441,1.64388585715e-05,1.00456778232,6.86738333092,...,72.0100525621,55.9897447645,103.017228091,25.9197942267,29.2536284986,97.8589524119,205.747338873,281.20586558,1.15046838218,0
16,Respiratory effects on humans caused by organi...,2.76694965206e-14,120.635818758,203.637822785,69.0571436452,24.5490330689,8.64950708812,0.00198274953583,15.5553686121,123.19345121,...,304.294828883,235.249664002,297.441393459,172.975741906,200.405846136,519.418182735,625.691444468,977.038019452,3.87644145962,0
17,Respiratory effects on humans caused by inorga...,0,564.593591442,915.451407411,255.430863002,223.05712293,105.235244736,0.689402447691,72.3508389726,2385.22836951,...,213.134059209,197.553399803,203.382560593,114.979528133,129.793192884,196.632844762,1797.59533717,1516.09828325,11.176968255,0
18,Damages to human health caused by climate chan...,9.51138942895e-12,103.527648623,149.015585124,47.8492422359,34.889471455,17.0689359166,0.104446275838,12.8301240137,400.32111911,...,5646.52433295,247.063105301,69.444047218,531.867900129,165.604520594,100.179340652,654.461251351,1216.14860719,13.2559070245,0
19,Damage to Ecosystem Quality caused by ecotoxic...,0,992940.198051,1253692.62925,536648.458073,153354.230623,70930.6810247,34.4830629582,109000.296075,888457.397712,...,38318247.0791,25076317.0935,58099920.6916,18451746.5014,20639473.1395,80755013.2526,94467140.1656,156567959.616,562870.129002,0


In [23]:
satellite_F.iloc[10:20]

,region,AT,AT.1,AT.2,AT.3,AT.4,AT.5,AT.6,AT.7,AT.8,...,WM.190,WM.191,WM.192,WM.193,WM.194,WM.195,WM.196,WM.197,WM.198,WM.199
10,Operating surplus: Remaining net operating sur...,0,135.600652856,366.32012529,621.057102504,87.7712448984,30.2112423977,1.0160481632,76.7774146546,81.6057857693,...,65.3549002087,33.2361305618,75.5964714341,12.8012525067,17.1774059697,88.4739508558,3249.66259066,2596.91649385,247.98260798,0
11,Employment: Low-skilled male,0,0.504325971453,1.02521709958,1.73437199088,0.117882592615,0.135305649918,0.000154971485262,0.290416316289,0.753954812361,...,1.09472417315,0.652596470282,1.113898987,0.352918040281,0.452633305684,34.6120090953,58.5689392392,30.3605244854,18.9250950769,0
12,Employment: Low-skilled female,0,0.243131905058,0.494249752399,0.836128199001,0.0568303457247,0.0652298756989,7.47106327538e-05,0.140007606659,0.363476164689,...,0.17192727627,0.105961865729,0.157576757281,0.0630621032416,0.0755427227672,11.4326823346,17.2892606505,9.61525018909,52.8200575214,0
13,Employment: Medium-skilled male,0,4.67686905151,9.50735515425,16.0837060692,1.09318472639,1.25475752274,0.00143712873085,2.6931769502,6.99180317446,...,6.19093802461,3.62132854625,6.30401226135,1.84316554556,2.42417397137,58.114102235,117.315099009,64.6062180642,9.74202565842,0
14,Employment: Medium-skilled female,0,3.91037147301,7.94918351776,13.4477285339,0.914021393738,1.04911383416,0.00120159601012,2.2517890071,5.84590831542,...,0.666444578173,0.394301426475,0.653770099942,0.233864728854,0.289029473537,15.6844336341,30.4172833172,17.0024949885,29.9024506451,0
15,Employment: High-skilled male,0,0.17241913554,0.350501572507,0.592947689189,0.0403017410649,0.0462583418523,5.29817043632e-05,0.0992876295005,0.257762329012,...,2.70606119891,1.53192337813,2.86463143921,0.711885291633,0.993925964331,34.8133201731,63.727330789,36.2231104043,1.45789252786,0
16,Employment: High-skilled female,0,0.0891483651878,0.181224909213,0.306580339634,0.0208377934324,0.0239176210896,2.73938986764e-05,0.0513361224417,0.133274593719,...,0.302288345281,0.168673860196,0.344942259522,0.0833545118252,0.117104219589,3.20256975264,6.89020996638,4.09463369653,1.39000136238,0
17,Employment hours: Low-skilled male,0,1.30437881344,2.65160142361,4.48574574298,0.304889228363,0.349951882475,0.000400815213781,0.751127071549,1.95001395764,...,2.97211160025,1.76765052407,3.02953634674,0.94416381603,1.21841673006,96.2969043772,162.082928969,84.1576261703,52.7440805335,0
18,Employment hours: Low-skilled female,0,0.628831596585,1.27831787786,2.16254559517,0.146984892945,0.168709272734,0.000193230117065,0.362112931285,0.940087632299,...,0.418918738167,0.255864845469,0.386160384721,0.147628746076,0.179570953185,31.7552448977,46.4063811175,26.4810983215,147.214355476,0
19,Employment hours: Medium-skilled male,0,12.0961625007,24.5896371335,41.5985823174,2.82739156217,3.24528027766,0.00371696159788,6.96558010796,18.0834627696,...,16.8096656788,9.81724462102,17.1489062796,4.93086250517,6.52881662618,161.171425558,319.151656492,177.466069347,27.1301502144,0


In [25]:
# number of meaningful rows
len(impacts_F['region'].tolist()[2:])

126

In [24]:
impacts_F['region'].tolist()[2:]

['Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
 'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
 'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
 'Damages to human health caused by climate change (H.A) | ECOINDICATOR 99 (H.A) | Damages to human health caused by climate change (H.A)',
 'Damage to Ecosystem Quality caused by ecotoxic emissions (H.A) | ECOINDICATOR 

In [26]:
satellite_F['region'].tolist()[2:]

['Taxes less subsidies on products purchased: Total',
 'Other net taxes on production',
 "Compensation of employees; wages, salaries, & employers' social contributions: Low-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: Medium-skilled",
 "Compensation of employees; wages, salaries, & employers' social contributions: High-skilled",
 'Operating surplus: Consumption of fixed capital',
 'Operating surplus: Rents on land',
 'Operating surplus: Royalties on resources',
 'Operating surplus: Remaining net operating surplus',
 'Employment: Low-skilled male',
 'Employment: Low-skilled female',
 'Employment: Medium-skilled male',
 'Employment: Medium-skilled female',
 'Employment: High-skilled male',
 'Employment: High-skilled female',
 'Employment hours: Low-skilled male',
 'Employment hours: Low-skilled female',
 'Employment hours: Medium-skilled male',
 'Employment hours: Medium-skilled female',
 'Employment hours: High-skilled male',
 'Employment ho

In [23]:
len(F_Y['region'].tolist()[2:])

126

In [21]:
F_Y['region'].tolist()[2:]

['Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
 'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
 'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
 'Damages to human health caused by climate change (H.A) | ECOINDICATOR 99 (H.A) | Damages to human health caused by climate change (H.A)',
 'Damage to Ecosystem Quality caused by ecotoxic emissions (H.A) | ECOINDICATOR 

In [16]:
S['region'].tolist()

['sector',
 'impact',
 'Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
 'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
 'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
 'Damages to human health caused by climate change (H.A) | ECOINDICATOR 99 (H.A) | Damages to human health caused by climate change (H.A)',
 'Damage to Ecosystem Quality caused by ecotoxic emissions

In [10]:
S['region'].tolist()

['sector',
 'impact',
 'Value Added',
 'Employment',
 'Employment hour',
 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
 'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
 'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
 'EPS | Damage Approach | EPS (Steen, 1999))',
 'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
 'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
 'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
 'Damages to human health caused by climate change (H.A) | ECOINDICATOR 99 (H.A) | Damages to human health caused by climate change (H.A)',
 'Damage to Ecosystem Quality caused by ecotoxic emissions

In [ ]:
S_Y = read_ecoinvent_txt(path_file_S_Y)
M = read_ecoinvent_txt(path_file_M)